In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os
from scipy import stats

In [ ]:
def Displ_func(frame_cur, frame_next):

    dr = []
    
    for particle in frame_cur['particle']:
        
        if (particle in frame_next['particle'].unique()):
            mask1 = frame_cur['particle'] == particle
            x1 = frame_cur[mask1]['x'].item()
            y1 = frame_cur[mask1]['y'].item()

            mask2 = frame_next['particle'] == particle
            x2 = frame_next[mask2]['x'].item()
            y2 = frame_next[mask2]['y'].item()

            dr.append( np.sqrt((x2 - x1)**2 + (y2 - y1)**2) )
    
    
    return dr
    

In [ ]:
def Assembly(frame):

    global data
    global max_frame
    
    dr = []
    
    frame_cur  = data[data['frame'] == frame]
    frame_next = data[data['frame'] == frame + 1]
    

    dr_cur = Displ_func(frame_cur, frame_next)
    dr.extend(dr_cur)
        
            
    if len(dr) > 0:        
        dr_mean = np.mean(dr)
    else:
        dr_mean = math.nan
    
    
    return dr_mean

In [ ]:
def read_data():
    
    file_path = 'data_out/backup/S6T34to18_t_corrected.csv'
    
    usecols = ['y', 'x', 'frame', 'particle']
    
    data = pd.read_csv(file_path, usecols=usecols)
    
    
    return data

In [ ]:
%%time



data = read_data()
max_frame = data['frame'].values[-1] + 1

DR = []
frames = []

for fr in range(max_frame - 1):
    
    dr_mean_cur = Assembly(fr)

    if math.isnan(dr_mean_cur):
        print('no common particles')
        continue
    else:
        DR.append(dr_mean_cur)
        frames.append(fr)
        
        print(f'fr = {fr} V')


In [ ]:
# convert to microns
px_to_micron = 0.06905
frame_to_sec = 1./60.

In [ ]:
time_sec  = [i*frame_to_sec for i in frames]
DR_micron = [i*px_to_micron*px_to_micron for i in DR]

In [ ]:
dict = {"time_sec": time_sec, "DR_micron": DR_micron}

df = pd.DataFrame(dict)

In [ ]:
# filter outliers

filter_outliers = df[np.abs(stats.zscore(df["DR_micron"])) < 2].all(axis=1)
df_filtered = df.loc[df.index.isin(filter_outliers.index)]

In [ ]:
# fit with the curve

from scipy.optimize import curve_fit


def fit_func(x, a, b, c):
    
    return a*x**2 + b*x + c


# curve fit
x = df_filtered.time_sec
y = df_filtered.DR_micron

popt, _ = curve_fit(fit_func, x, y)
a, b, c = popt

x_fit = np.arange(min(x), max(x), 1.e-2)
y_fit = fit_func(x_fit, a, b, c)

In [ ]:
def highResPoints(x, y, factor=10):
    '''
    Take points listed in two vectors and return them at a higher
    resultion. Create at least factor*len(x) new points that include the
    original points and those spaced in between.

    Returns new x and y arrays as a tuple (x,y).
    '''

    # r is the distance spanned between pairs of points
    r = [0]
    for i in range(1,len(x)):
        dx = x[i]-x[i-1]
        dy = y[i]-y[i-1]
        r.append(np.sqrt(dx*dx+dy*dy))
    r = np.array(r)

    # rtot is a cumulative sum of r, it's used to save time
    rtot = []
    for i in range(len(r)):
        rtot.append(r[0:i].sum())
    rtot.append(r.sum())

    dr = rtot[-1]/(NPOINTS*RESFACT-1)
    xmod=[x[0]]
    ymod=[y[0]]
    rPos = 0 # current point on walk along data
    rcount = 1 
    
    while rPos < r.sum():
        x1,x2 = x[rcount-1],x[rcount]
        y1,y2 = y[rcount-1],y[rcount]
        dpos = rPos-rtot[rcount] 
        theta = np.arctan2((x2-x1),(y2-y1))
        rx = np.sin(theta)*dpos+x1
        ry = np.cos(theta)*dpos+y1
        xmod.append(rx)
        ymod.append(ry)
        rPos+=dr
        while rPos > rtot[rcount+1]:
            rPos = rtot[rcount+1]
            rcount+=1
            if rcount>rtot[-1]:
                break

    return xmod,ymod


#CONSTANTS
NPOINTS = len(x_fit)
# COLOR='blue'
RESFACT=10
MAP='jet_r'#'winter' # choose carefully, or color transitions will not appear smoooth

# # create random data
# np.random.seed(101)
# x = np.random.rand(NPOINTS)
# y = np.random.rand(NPOINTS)

# fig = plt.figure()
# ax1 = fig.add_subplot(221) # regular resolution color map
# ax2 = fig.add_subplot(222) # regular resolution alpha
# ax3 = fig.add_subplot(223) # high resolution color map
# ax4 = fig.add_subplot(224) # high resolution alpha

# Choose a color map, loop through the colors, and assign them to the color 
# cycle. You need NPOINTS-1 colors, because you'll plot that many lines 
# between pairs. In other words, your line is not cyclic, so there's 
# no line from end to beginning
# NPOINTS

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))


# all data
# ax.plot(df.time_sec, df.DR_micron, marker='o', ms=5, ls='-', label='S6, all')
# without outliers
ax.plot(df_filtered.time_sec, df_filtered.DR_micron, marker='o', ms=5, ls='none', color='gray')
# fit curve
# ax.plot(x_fit, y_fit, marker='*', ms=5, ls='--', label='fit')



cm = plt.get_cmap(MAP)
ax.set_prop_cycle('color', [cm(1.*i/(NPOINTS-1)) for i in range(NPOINTS-1)])

for i in range(NPOINTS-1):
    ax.plot(x_fit[i:i+2], y_fit[i:i+2])


ax.set_xlabel(r'$\tau$ [sec]', fontsize=16)
ax.set_ylabel(r'$\langle \Delta r \rangle$ [$\mu$m]', fontsize=16)

# ax.set_xscale('log')
# ax.set_yscale('log')
# plt.legend (fontsize=14)

In [ ]:
# write to file
DIR = 'data_out/'

file_DR = DIR + 'Displacements_S6T34to18.dat'

df.to_csv(file_DR, index=False)